<a href="https://colab.research.google.com/github/AraiKensuke/GCoh/blob/master/simulations/simulated_multichannel_EEG_Gcoh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
#########  clone and install necessary packages from github repositories

import sys
import importlib

if importlib.util.find_spec("clustering_GCoh") is None:
  !git clone https://github.com/AraiKensuke/clustering_GCoh.git

if importlib.util.find_spec("mne") is None:
  !pip install mne

sys.path.insert(1, "/content/clustering_GCoh")    #  add these to searchpath for python modules


In [19]:
import GCoh



ModuleNotFoundError: ignored

In [ ]:
#####  Generate simulated EEG data.

import numpy as _N
import os

import GCoh.datconfig as datconf
import GCoh.simulations.sim_utils as _su

#  grpA and grpB   can coexist
#  grpA and grpC   can coexist
#  grpB and grpD   can coexist

grpA = _N.array([1, 2, 5])
grpB = _N.array([3, 4, 8])
grpC = _N.array([7, 8, 9])
grpD = _N.array([1, 6, 7])
grpA_phase_diff = _N.array([0, -2, 5])
grpB_phase_diff = _N.array([0, 13, 9])
grpC_phase_diff = _N.array([0, 1, 1])
grpD_phase_diff = _N.array([0, 0, 0])

dt   = 0.005
nChs = 10
N    = 10000   #(50 s)
f    = 20      #  oscillation frequency 
amp  = 0.995

#  up to 4 groups of
rhythms = _N.zeros((nChs, N))
pknzs   = _N.zeros((nChs, N))
tau     = 0.15    #  timescale of AR1

######  we generate simulated EEG by generating nChs of independent oscillatory
######  signals.  
for nc in range(nChs):
    rhythms[nc] = _su.AR2(f, 0.98, N, dt)
    pknzs[nc] = _su.AR1(tau, N, dt)
EEG     = _N.array(rhythms)

#mix(dt, EEG, rhythms, pknzs, grpA, 10, 15)
_su.mix(dt, EEG, rhythms, pknzs, grpB, grpB_phase_diff, 18, 45)
#mix(dt, EEG, rhythms, pknzs, grpD, 18, 45)
#mix(dt, EEG, rhythms, pknzs, grpC, 28, 34)
#mix(dt, EEG, rhythms, pknzs, grpD, 40, 45)

outfn = _su.generate_artfctrmvd_filename()
_N.savetxt(outfn, EEG.T, fmt=("%.4f " * nChs))
print("saved %s"  % outfn)








In [ ]:
import numpy as _N
import scipy.signal as _ssig
import mne.time_frequency as mtf
import mne
import pickle
import sys

import GCoh.chronux_py.chronux_funcs as _chrf
import preprocess_ver

import GCoh.datconfig as datconf
import GCoh.windowed_gcoh as _w_gcoh

dataset =    datconf._SIM
Fs      = 200

artrmv_ver = 1
gcoh_ver    = 1

wnd, slideby      = preprocess_ver.get_win_slideby(gcoh_ver)

rm_chs = []
ch_w_CM = _N.arange(10)

X_cm    = EEG
ch_picks = _N.setdiff1d(ch_w_CM, _N.array(rm_chs))
arr_ch_picks = _N.array(ch_picks)

info = mne.create_info(ch_names=(_N.array(ch_names)[ch_picks]).tolist(), ch_types=(_N.array(ch_types)[ch_picks]).tolist(), sfreq=Fs)

datconf.set_montage(dataset, info)

dpss_bw=7
f, findx, Ctot, Cvec = _w_gcoh.windowed_gcoh(Fs, wnd, slideby, X_cm, ch_w_CM, ch_picks, info, dpss_bw=dpss_bw)

pkldat = {"VEC" : Cvec, "Cs" : Ctot, "fs" : f[findx], "chs_picks" : arr_ch_picks, "dpss_bw" : dpss_bw}

